In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from feature_selection import ReliefF
from feature_extraction import ConventionalFeatures, CoutinhoFeatures, MurugaiahFeatures, ZhangFeatures

%config InlineBackend.figure_format = 'svg'

In [2]:
df = pd.read_parquet('data/dataset.parquet')
df.drop(columns=["strain", "date", "date_submitted"], inplace=True)

In [3]:
df

,pangolin_lineage,sequence
0,BA.1.1.1,AACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAAT...
1,BA.1.1,AGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCT...
2,BA.1.1,AGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCT...
3,BA.1.15,AGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCT...
4,BA.1.1,ACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAA...
...,...,...
363,AY.127,AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...
364,AY.36,AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...
365,B.1.617.2,AGATCTGTTCTCTAAACGAACTTTAAAATCTGGGTGGCTGTCACTC...
366,AY.121,AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...


In [4]:
murugaiah_features = MurugaiahFeatures().extract_features(df.sequence.values)
print(murugaiah_features.shape)

(368, 118)


In [11]:
relieff = ReliefF(n_instances=300, n_neighbors=10)
n_features_to_select = 30
is_binary_features = False

In [ ]:
selected_murugaiah_features = relieff.fit_transform(
    X=murugaiah_features, 
    y=df.pangolin_lineage.values, 
    n_select=30, 
    is_discrete=False,
    n_iterations=100,
)

In [12]:
selected_murugaiah_features.shape

(368, 30)

In [15]:
np.argsort(relieff.feature_importances_)

array([ 88,  48,  36,  30,  84,  80,  50,   1,  54,  78,  37,  64,  70,
        60,  56,  85,  57,  62,  82,  29,  34,  53,  42,  26,  17,  81,
        76,  24,  28,  35,  86,  13,  74,  63,  33,  58,  43,  55,  89,
        41,  83,  77,  38,  72,  20,  39,  68,  75,  40,  66,  18,  87,
        71,  67,   8,  31,  79,  15,  27,  16,  14,  19,  59,  21,  22,
        23,  46,  10,  11,  73,  51,  32,  69,  12,  49,   7,   9,  65,
       111,   5,   4, 100,  47,   6,  61,   3,  44,  45,  52, 110, 101,
        93,  25, 116, 104, 107,  95,  97,  91, 102, 115,  98,   2, 106,
       109,  92, 112, 105, 108,  96,  94, 114,  99,  90, 103, 113,   0,
       117])